In [ ]:
import requests
import csv
import json
import os
from datetime import datetime

# Insert your VT API key here
api_key = "You API Key Here"

# Used as input for educated guesses on the hash type
file_hash_type_length = {32: "MD5", 40: "SHA1", 64: "SHA256", 128: "SHA512"}

# Hash types returned by external VT
file_hash_types = ["md5", "sha1", "sha256", "ssdeep"]

# Define base directories for different options
base_directories = {
    "Dir1": "path/to/your/directory/files/",
    "Dir2": "path/to/your/directory/files/",
    "Dir3": "path/to/your/directory/files/"
}

def check_masshashes(directory, output_csv, subdirectory_name):
    # Ensure the directory exists
    if not os.path.isdir(directory):
        print(f"Directory does not exist: {directory}")
        return
    
    stack = []
    print(f"Reading files from directory: {directory}")
    
    # Traverse the directory and process files
    for root, dirs, files in os.walk(directory):
        for file in files:
            hash_with_extension = file.strip()
            hash_only, file_extension = os.path.splitext(hash_with_extension)  # Remove the file extension
            file_extension = file_extension.lstrip('.')  # Remove leading dot from the extension
            stack.append((hash_only, file_extension))

    if not stack:
        print("No files found to process.")
        return
    
    file_exists = os.path.isfile(output_csv)
    with open(output_csv, 'a', newline='') as csv_output:
        csv_writer = csv.writer(csv_output)
        if not file_exists or os.stat(output_csv).st_size == 0:
            csv_writer.writerow(['Date', 'Hash Type', 'Value', 'File Type', 'Result'])
        while stack:
            gethash, file_extension = stack.pop()
            hash_type = determine_hash_type(gethash)
            if hash_type:
                x = printresult(gethash, hash_type)
                if x:
                    score = x.get('positives', 0)
                    if score >= 1:
                        result = 'Malicious'
                        csv_writer.writerow([subdirectory_name, hash_type, gethash, file_extension, result])
                        print(f"Writing result for hash {gethash} to file with file type {file_extension}.")
                else:
                    print(f"No result found for hash {gethash}.")
            else:
                print(f"Unknown hash type for: {gethash}")
                continue
    print(f"\nDone Lookup From VirusTotal. Check Your Results In {output_csv}.")

def determine_hash_type(hash):
    return file_hash_type_length.get(len(hash), None)

def vt_getresult(hash, hash_type):
    headers = {
        "Accept-Encoding": "gzip, deflate",
        "User-Agent": "gzip, My Python requests library example client or username"
    }
    params = {'apikey': api_key, 'resource': hash}
    response = requests.post(f'https://www.virustotal.com/vtapi/v2/file/report', params=params, headers=headers)
    try:
        json_response = response.json()
        return json_response
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return None

def printresult(hash, hash_type):
    getresult = vt_getresult(hash, hash_type)
    return getresult

def list_subdirectories(base_directory, year_month):
    subdirectories = [
        name for name in os.listdir(base_directory) 
        if os.path.isdir(os.path.join(base_directory, name)) and name.startswith(year_month)
    ]
    return subdirectories

while True:
    # Prompt for the directory type
    directory_type = input("\033[1mEnter the directory type (Dir1, Dir2, Dir3) or type 'exit' to quit:\033[0m").strip().lower()

    if directory_type == 'exit':
        print("\033[1mExiting the program.\033[0m")
        break

    if directory_type not in base_directories:
        print("\033[1mInvalid directory type. Please enter 'Dir1', 'Dir2', or 'Dir3'.\033[0m")
        continue

    base_directory = base_directories[directory_type]

    # Prompt for the specific month (YYYY-MM)
    year_month = input("\033[1mEnter the year and month to process (format YYYY-MM) or type 'exit' to quit:\033[0m").strip()

    if year_month.lower() == 'exit':
        print("\033[1mExiting the program.\033[0m")
        break

    # List subdirectories that match the specified year and month
    subdirectories = list_subdirectories(base_directory, year_month)

    if not subdirectories:
        print(f"No subdirectories found for {year_month} in {base_directory}. Exiting.")
        continue

    # Create the folder structure based on the directory type and the year and month
    output_base_directory = os.path.join("path/to/your/directory/files/File Analysis", directory_type)
    output_directory = os.path.join(output_base_directory, year_month)
    os.makedirs(output_directory, exist_ok=True)

    # Adjust output CSV file name to include "Malicious-File"
    output_csv = os.path.join(output_directory, f"{year_month}-Malicious-File.csv")

    for subdirectory in subdirectories:
        # Combine base directory with the subdirectory name
        directory = os.path.join(base_directory, subdirectory)
        # Check multiple hashes from files in the specified directory and save results to the same file
        check_masshashes(directory, output_csv, subdirectory)

    print(f"Processing complete for {year_month}. Results saved to {output_csv}.")